In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
# this class is for creating instances of Nodes 
class TreeNode:
        
    def __init__(self,level):
        self.__level=level          # level of the node in tree structure
        self.splittingFeature=""    # on which feature the node will be splitted, if no further split then its value is null
        self.__entropy=0.0          # entropy of the node
        self.__gainRatio=-1         # gain ratio of the node
        self.childList=[]           # this is list for storing the child nodes of the current node
        self.result=""              # used for storing the class which will be predicted on the basis of majority
  


    def addChild(self,node,data):   # method to add the child node and its data (splitted feature's value)
        self.childList=self.childList+[(node,data)]              # appending the list
#         self.childList.append((node,data))


    def setSplittingFeature(self,f):       # method to set the splitting feature value
        self.splitting_feature=f
    
    
    def printCount(self,y):                # this method is used for printing the unique value (with their count) present 
                                           # in the output i.e. Y corresponding to data of current node
        
        dict={"0":"setosa","1":"versicolor","2":"virginica"} # dictionary storing the actual value of the output
        
        a=y.value_counts(ascending=True)                     # finding the unique values with their count in output
        
        for i in range(a.shape[0]):                          # printing the unique values with their count
            print("Count of ",str(a.index[i]),"(",dict[str(a.index[i]).strip()],")",a.values[i])
        
        self.result=str(a.index[a.shape[0]-1]) # setting class to be predicted for node to class/value whose count is maximum
            
            
    def setEntropy(self,entropy):           # method to set entropy 
        self.__entropy=entropy
    
    
    def setGainRatio(self,gainRatio):       # method to set gain ratio 
        self.__gainRatio=gainRatio
    
    
    def printDetails(self,y):               # method to print the details (that are various fields) of the node
        
        dict={"0":"setosa","1":"versicolor","2":"virginica"}           # dictionary storing the actual value of the output
        print("Level",self.__level)
        self.printCount(y)
        print("Current Entropy  is =",self.__entropy)
        
        # checking whether the node is splitted further or not
        
        if(self.splittingFeature!=""):     
            print("Splitting on feature",self.splittingFeature,"with gain ratio",self.__gainRatio)
        else:
            print("Reached leaf Node")
        print("Class to be predicted",self.result,"(",dict[str(self.result).strip()],")")

In [3]:
import math

def gain(x, y, f):                       # method to calculate the gain ratio, 
                                         # Arguments are:- x : pandas dataframe, y : pandas series, f:list containing features  
    
    labels=getLabels(x[f])               # getting unique labels corresponding to the feature 
    x["y"]=y                             # adding new column i.e output y in x dataframe

    split_info_ = split_info(x[f])       # calling function to calculate the split info of the feature
    
    a=x["y"].value_counts()              # finding the unique values of the output data
    length_y=x.shape[0]                 
    
    entropy=info(a,length_y)        # for calculating the information of the feature i.e entropy
    
    # Calculating the Info gain
    
    new_info=0
    for i in labels:                     # used for info gain after splitting to different nodes
        new_x=x[x[f]==i]
        a1=new_x["y"].value_counts()
        length_y1=new_x.shape[0]
        
        info_i=info(a1,length_y1)
        
        new_info+=float(new_x.shape[0]*info_i)/float(x.shape[0])
        
    info_gain=entropy-new_info                           # info gain
    
    if(split_info_==0):
        return info_gain
    
    gain_ratio=info_gain/split_info_                     # gain-ratio

    return (entropy,gain_ratio)
    
    
def split_info(x):                                       # method to calculate split ; Argument x : pandas dataframe
    split_info=0                                           
    a=x.value_counts()                                   # for getting unique values with their count
    
    for i in range(a.shape[0]):
        p=float(a.values[i])/float(x.shape[0])           # caculating the probability of each splitted node
        log=math.log(p,10)
        split_info += p*log
    split_info= (-1)*split_info
    
    return split_info

def info(a,length_y):                                    # method to calculate information corresponding to a node
                                                         # Arguments :- a : pandas series, length_y: int

    info_i=0
        
    for i in range(a.shape[0]):
        p=float(a.values[i])/float(length_y)
        log=math.log(p,10)
        info_i += p*log
        
    info_i= (-1)*info_i
    
    return info_i
    
def getLabels(x):                                        # method for finding each unique label corresponding to the feature
                                                         # x : pandas dataframe
    
    a=x.value_counts()                                   # for getting unique values with their count
    length=a.shape[0]
    labels=[]
    for i in range(a.shape[0]):
        labels.append(a.index[i])
    return labels


def decisionTree(x, y, level, features):                # Method to build the decision tree
                                                        # Arguments:- x: pandas dataframe, y : pandas series
                                                        #   level: int, features: list containing features' name
    print()
    
    node=TreeNode(level)                                # Creating the Tree Node object 
    
    a=set()                                             # creating set for finding the unique values of y
    
    for i in range(y.shape[0]):                         # adding the value of output i.e y in the set
        a.add(y.iloc[i])      
        
    if(len(a)==1):                      # if the set contains single value i.e. their is only one value of output possible 
                                        # means pure node is achieved
        node.printDetails(y)            # calling node method for printing the details of the node
        return node                     # returning as further no split is possible

    elif(len(features)<=0):             # checking whether the features are available 
                                        # so no more features to split upon as features array is empty
            
        a=y.value_counts()              # calculating the different labels with thier count of the output corresponding to the  
                                        # node as this may not be a pure node and if not pure node, majority will decide output 
        
        length_y=x.shape[0]

        info_feature=info(a,length_y)                   # calculating the information of the feature i.e the entropy 
        node.setEntropy(float(info_feature))            # setting the entropy value of the node
        node.printDetails(y)                            # calling node's method for printing the details of the node
        return node
    
    
    #### Finding the best feature to split upon, feature with maximum gain
    
    max_gain=-1
    max_entropy=-1
    best_feature=""
    
    for f in features:                  # calculating gain ratio of each feature
        g1=gain(x, y, f)
        g=g1[1]
        entropy=g1[0]
        if(g>max_gain):                 # updating the best feature according to the maximum gain
            best_feature=f
            max_gain=g
            max_entropy=entropy
            
    features.remove(best_feature)       # removing the current feature so that it may not be used again for splitting
    
    #### setting different fields of the current Node
    node.setSplittingFeature(best_feature)
    node.splittingFeature=best_feature
    node.setEntropy(max_entropy)
    node.setGainRatio(max_gain)
    node.printDetails(y)                # calling node's method for printing the details of the node
    
    
    #### Recursion to build the complete tree
    
    labels=getLabels(x[best_feature])         # getting the different labels corresponding to the selected feature
    
    x["y"]=y                                  # adding the output column i.e. y to x dataframe so that it is easy to split the 
                                              # output y corresponding to the split of x dataframe  
    
    for i in labels:                          # iterating over each label so to build new nodes corresponding to each label
        
        new_x = x[x[best_feature]==i]   # getting x dataframe where the value of feature (column) is equal to the current label
        new_y = new_x["y"]                    # getting the new y for corresponding x dataframe
        new_x.drop('y',axis=1,inplace=True)   # droping the output column from 'new_x' dataframe as we have output in 
                                              #'new_y' variable
    
        #### recursive step to build nodes
        node.addChild(decisionTree(new_x, new_y, level+1, features),i)
    
    x.drop('y',axis=1,inplace=True)           # droping the output column form x dataframe as we have output stored in y
    return node                               # this method returns the node
           

In [4]:
def fit(x,y,features):                          # method for training the algorithm using training input (x : pandas dataframe)
                                                # and output (y : pandas series),also the list of features (features' list)
        
    return decisionTree(x, y, 0, features)      # building decision tree
    
    
def predictValue(node,x):                        # method to predict the output corresponding to a single row of x dataframe
                                                 # Arguments:- node: TreeNode, x : single row of pandas dataframe 
    
    if (len(node.childList)==0):                 # base case, when node is not further splitted
        return node.result                       # so we return the majority value of the node as a result, 
                                                 # this is stored in node's field "result"
        
    x_data=x[node.splittingFeature]              # getting the column corresponding to the splitted feature
    x_data=x_data.iloc[0]                        # getting the value in the column, as this is a single row
    
    for i in range(len(node.childList)):         # for finding to which label the value of our data ('x_data' variable) belongs
                                                 # by looking at each child nodes (labels corresponding to splitting feature) of
                                                 # the node 
                
        value=node.childList[i]                  # getting the label 
        curr_node=value[0]                       # getting the node of the label
        curr_node_data=value[1]                  # getting the value of the label
                    
        if(x_data==curr_node_data):              # comparing our data with the label value
            return predictValue(curr_node,x)     # using recursion, and sending the node of the label whose value matches 
                                                 # our data
    
def predict(node,x):                     # method to predict the output corresponding to dataframe x
                                         # Arguments:- node: TreeNode, x : complete pandas dataframe

    y=[]                                 # used for storing output corresponding to each row of x
    
    for i in range(x.shape[0]):          # finding output corresponding to each row of x
        a=x.iloc[i:]                     # getting a row of x
        result=predictValue(node,a)      # predicting the value corresponding to this row by calling the 'predictValue' function
        y.append(result)                 # adding the result to our output array
    Y=np.array(y)                        # converting the array to numpy array
    y = pd.Series(Y)                     # converting the numpy array to pandas series
    return y
        
def score(y_pred,y_true):                   # method for calculating the score 
                                            # Arguments:- y_pred: pandas series, y_true: pandas series 
    count=0                                 # stores the number of values where the output predicted is same as actual output
    for i in range(y_true.shape[0]):        # iterating over each row of predicted and actual output
        a=y_true.iloc[i]                    # value of the actual output for the current row
        b=y_pred.iloc[i]                    # value of the predicted output for the current row
        
        if(str(a).strip()==str(b).strip()):   # if the actual and predicted output are same the count is increased by 1
            count+=1
            
    score=count/y_true.shape[0]         # calculating the score, score = (no of correct values predicted)/(total no of values)
    return score
     

In [5]:
iris=datasets.load_iris()                   # getting the iris data
x_train,x_test,y_train,y_test= train_test_split(iris.data,iris.target,random_state=1) # splitting into testing and training data

import warnings
warnings.filterwarnings('ignore')
BOLD = '\x1b[1;30;30m'
END = '\x1b[0m'

def get_x(x_train,boolean_Data):                    # to get x which can be passed to our decision tree
                                                    # boolean_Data is for printing purposes whether to print x dataframe or not
                                                    # Arguments:- x: numpy n-dim array, boolean_Data: True/False
    
    features=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']    # feature's name 
    x = pd.DataFrame(x_train,columns=features)        # converting the x data to pandas dataframe
    
    if(boolean_Data):
        # printing the iris data
        print(BOLD+"----------------------------------- Actual Iris Data -----------------------------------"+END)          
        print(x)
        print()
    
    def fun(s):                             # method to convert string value (contains real number) to float 
                                            # Argument:- s: string
        s=str(s)
        return float(s)

    def converting_to_classes(x):           # method to convert continous data into discrete data, attribute is a pandas series
                                            # Argument x: pandas dataframe
        
        b=x.apply(fun)                      # converting string value to float 

        a=b.copy()
        a=a.to_numpy(dtype=float)           # converting the pandas series to numpy array
        a.sort()                              # sorting the data of numpy array
        size=a.shape[0]                     # size of the numpy array
        
        #### the data (contionus) is splitted into 4 descrete values for which 3 boundaries are required
        first_split = a[int(size/4)]            # first boundary 
        second_split = a[int((2*size)/4)]       # second boundary 
        third_split = a[int((3*size)/4)]        # third boundary 
        
        result_array=[]                 # stores the discrete value (4 possibilites) corresponding to the each continous data
        
        for i in range(b.shape[0]):     # iterating over each row of the pandas series
            
            if(b[i]<=first_split):              # before first boundary, so the continous value is converted to class "0"
                result_array.append("class 0")
            
            elif (first_split<b[i] and b[i]<=second_split):  # between first and second boundary, so the continous value is 
                                                             # converted to class "1"
                result_array.append("class 1")
                
            elif (second_split<b[i] and b[i]<=third_split):  # between second and third boundary, so the continous value is 
                                                             # converted to class "2"
                result_array.append("class 2")
                
            else:                               # after fourth boundary, so the continous value is converted to class "3"
                result_array.append("class 3")
        result_array = np.array(result_array)   # result array is converted to numpy array     
        result_array = pd.Series(result_array)  # result array is converted to pandas series 

        return result_array

    #### Continous data of each column is converted to discrete classes
    x["sepal_length"]=converting_to_classes(x["sepal length (cm)"])          
    x["sepal_width"]=converting_to_classes(x["sepal width (cm)"])
    x["petal_length"]=converting_to_classes(x["petal length (cm)"])
    x["petal_width"]=converting_to_classes(x["petal width (cm)"])
    x.drop(features,axis=1,inplace=True)                                     # continous data is droped
    
    if(boolean_Data):
        # printing the iris data after converting the Continous Data to Discrete Classes
        print(BOLD+"----------- Iris Data after converting the Continous Data to Discrete Classes -----------"+END)
        print(x)
        print()
    
    return x
x = get_x(x_train,True)                     # getting the x dataframe which can be passed to decision tree
y = pd.Series(y_train)                      # converting the y data to pandas series, so that it can be passed to decision tree


features=["sepal_length","sepal_width","petal_length","petal_width"]     # features to be passed to decision tree


----------------------------------- Actual Iris Data -----------------------------------
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  6.5               2.8                4.6               1.5
1                  6.7               2.5                5.8               1.8
2                  6.8               3.0                5.5               2.1
3                  5.1               3.5                1.4               0.3
4                  6.0               2.2                5.0               1.5
..                 ...               ...                ...               ...
107                6.3               2.8                5.1               1.5
108                6.4               3.1                5.5               1.8
109                6.3               2.5                4.9               1.5
110                6.7               3.1                5.6               2.4
111                4.9               3.6             

In [6]:

root = fit(x, y, features)                        # using the x and y data to build the decision tree, fit method will return 
                                                  # the root node of the tree


Level 0
Count of  1 ( versicolor ) 34
Count of  0 ( setosa ) 37
Count of  2 ( virginica ) 41
Current Entropy  is = 0.47584404860389007
Splitting on feature petal_length with gain ratio 0.5975745899049689
Class to be predicted 2 ( virginica )

Level 1
Count of  0 ( setosa ) 33
Current Entropy  is = 0.0
Reached leaf Node
Class to be predicted 0 ( setosa )

Level 1
Count of  1 ( versicolor ) 13
Count of  2 ( virginica ) 15
Current Entropy  is = 0.2999211575627872
Splitting on feature petal_width with gain ratio 0.2865660631216426
Class to be predicted 2 ( virginica )

Level 2
Count of  2 ( virginica ) 8
Count of  1 ( versicolor ) 11
Current Entropy  is = 0.2955936308119856
Splitting on feature sepal_length with gain ratio 0.2720812438497109
Class to be predicted 1 ( versicolor )

Level 3
Count of  1 ( versicolor ) 6
Count of  2 ( virginica ) 7
Current Entropy  is = 0.2997438305836322
Splitting on feature sepal_width with gain ratio 0.13702530817459677
Class to be predicted 2 ( virginica 

In [7]:
y_pred=predict(root,x)                            # predicting the output corresponding the x, note x here is same as training x
                                                  # but is already converted to required form using 'get_x' method
print("SCORE of training data",BOLD,score(y_pred,y),END)   # printing the training data score by calling the 'score' method


#### Checking the algorithm by testing it on the testing data

x1 = get_x(x_test,False)  # getting x_test which can be passed to the decision tree (converting continous data to discrete data)
y1 = pd.Series(y_test)    # actual output is converted pandas series

y_pred_test=predict(root,x1)    # predicting the output of the testing input by passing the root of decision tree and input data
print("SCORE of testing data",BOLD,score(y_pred_test,y1),END)  # printing the testing data score

SCORE of training data  0.9285714285714286 
SCORE of testing data  0.8157894736842105 


Both training and testing score are good, so the decision tree algorithm is performing well, also splitting the continous data into 4 classes assures that decision tree is not too big or too short, so, in descent amount of time the prediction is done.